In [1]:
from riotwatcher import LolWatcher, ApiError
from riot_api import key
import pandas as pd
from datetime import datetime

region = 'euw1'

lol = LolWatcher(key)

In [2]:
me = lol.summoner.by_name(region,'Katiounette')
puuid = me['puuid']

In [3]:
matchs = lol.match.matchlist_by_puuid(region,puuid, count = 100)
i = 100
while (lol.match.matchlist_by_puuid(region,puuid, count = 100, start = i) != []):
    matchs.extend(lol.match.matchlist_by_puuid(region,puuid, count = 100, start = i))
    i = i + 100

In [4]:
matchData = pd.DataFrame({'playerID' : [],'matchID' : [], 'duration (s)' : [], 'start' : [], 'end' : [], 'championName' : [], 'championId' : [], 'pings' : [], 'ePings' : []})
matchData

,playerID,matchID,duration (s),start,end,championName,championId,pings,ePings


In [7]:
for j,i in enumerate(matchs):
    p = lol.match.by_id(region,i)
    for player in p['info']['participants']:
        if player['puuid'] == puuid:
            cName = player['championName']
            cId = player['championId']
            ePing = player['enemyMissingPings']
            pings = player["allInPings"] + player["assistMePings"] + player["baitPings"] + player["basicPings"] + player["commandPings"] + player["dangerPings"] + player["getBackPings"] + player["holdPings"] + player["needVisionPings"] + player["onMyWayPings"] + player["pushPings"] + player['visionClearedPings'] + player['enemyMissingPings'] + player['enemyVisionPings']
    matchData.loc[j,:] = [puuid,p['metadata']['matchId'],p['info']['gameDuration'], p['info']['gameStartTimestamp'], p['info']['gameEndTimestamp'], cName, cId, pings, ePing]
    break

In [8]:
matchData

,playerID,matchID,duration (s),start,end,championName,championId,pings,ePings
0,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6357952023,1536.0,1.681327e+12,1.681328e+12,Tristana,18.0,2.0,0.0


In [9]:
for i in range(matchData.shape[0]):
    matchData.loc[i,'start'] = pd.Timestamp(datetime.fromtimestamp(matchData.loc[i,'start']/1000))
    matchData.loc[i,'end'] = pd.Timestamp(datetime.fromtimestamp(matchData.loc[i,'end']/1000))
    matchData.loc[i,'weekDay'] = matchData.loc[i,'start'].day_name()
    matchData.loc[i,'startHour'] = matchData.loc[i,'start'].hour

In [10]:
matchData

,playerID,matchID,duration (s),start,end,championName,championId,pings,ePings,weekDay,startHour
0,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6357952023,1536.0,2023-04-12 21:14:32.594000,2023-04-12 21:40:09.131000,Tristana,18.0,2.0,0.0,Wednesday,21.0
